# This script shows how to get data from Wikipedia and clean it.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
#!pip install geopy 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
!pip install folium
import folium
import json
import requests
from pandas.io.json import json_normalize

!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
geosp_data_frame = pd.read_csv('Geospatial_data.csv')

     |████████████████████████████████| 94 kB 5.8 MB/s  eta 0:00:01
--2021-03-16 17:34:45--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 52.116.121.148, 52.116.127.82
Connecting to cocl.us (cocl.us)|52.116.121.148|:80... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://cocl.us/Geospatial_data [following]
--2021-03-16 17:34:45--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|52.116.121.148|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-03-16 17:34:47--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [follow

Get data from the link and retreave the coumns:

In [2]:
                                                                                                            
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'


dfs = pd.read_html(url)
df = dfs[0]                                                                                                        
data_frame = df[['Postal Code','Borough','Neighbourhood']]


Clean the data in the set:

In [3]:
data_frame.drop(data_frame[data_frame['Borough'] == 'Not assigned'].index, inplace = True) 
data_frame.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Merge the dataframes:

In [4]:
df_merge = data_frame.merge(geosp_data_frame, on="Postal Code", how = 'inner')
df_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


I decided to plot the ma of toronto and query data from Foursquare.

In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer_test")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [6]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_merge['Latitude'],df_merge['Longitude'],df_merge['Borough'],df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto


Insert your foursquare access information.

In [7]:


CLIENT_ID = '4AQWJX5KU2Z1N45M3TNVKEKACLPF5E3YINDJCOZU5CLHNVKD' # your Foursquare ID
CLIENT_SECRET = 'LDY3T4DOSDUF0H4J3JS2UFES2XAEPHZPS5U0D1CSGV152YUI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


In [8]:


neighbourhood_latitude = df_merge.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_merge.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = df_merge.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

Now we categorize venues and filter data:

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items'] 
nearby_venues = pd.json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
df_merge_small=df_merge[df_merge['Neighbourhood'].str.contains("Toronto")]

toronto_venues = getNearbyVenues(names=df_merge_small['Neighbourhood'],
                                   latitudes=df_merge_small['Latitude'],
                                   longitudes=df_merge_small['Longitude']
                                  )

East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
North Toronto West, Lawrence Park
University of Toronto, Harbord
New Toronto, Mimico South, Humber Bay Shores
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto


Let's speed up our table work using this encoding:

In [27]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

num_top_venues = 10

for hood in toronto_grouped ['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()



----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
----East Toronto, Broadview North (Old East York)----
----Harbourfront East, Union Station, Toronto Islands----
----New Toronto, Mimico South, Humber Bay Shores----
----North Toronto West, Lawrence Park----
----Toronto Dominion Centre, Design Exchange----
----University of Toronto, Harbord----


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Business reply mail Processing Centre, South C...",Yoga Studio,Brewery,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Comic Shop
1,"East Toronto, Broadview North (Old East York)",Intersection,Park,Convenience Store,French Restaurant,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
2,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Brewery,Fried Chicken Joint,Restaurant,Italian Restaurant,Sporting Goods Shop
3,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Pizza Place,Pet Store,Cosmetics Shop,Restaurant,Café,Mexican Restaurant,Fast Food Restaurant,Liquor Store,Pharmacy
4,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Gym / Fitness Center,Restaurant,Chinese Restaurant,Salon / Barbershop,Café


## The above table can give a clear picture of where to open a restaurant taking into account the options already available in the area. 
You can see that in New Toronto area the most common businesses are restaurants or cafes, but in the first row in the table raustaurant is only on place 9. 